In [0]:
from pyspark.sql.types import *
import pyspark.sql.functions as F
from elasticsearch import Elasticsearch, helpers
from pyspark.sql.functions import col, split
from pyspark.sql.types import ArrayType, IntegerType
from pyspark.sql.types import *
import pyspark.sql.functions as F
from elasticsearch import Elasticsearch, helpers
from pyspark.sql.functions import col, split, randn
from pyspark.sql.types import ArrayType, IntegerType
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression, LogisticRegressionModel
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import BinaryClassificationMetrics, MulticlassMetrics
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np 
from pyspark.sql.window import Window
import pickle
from collections import defaultdict


ES_HOST= '10.0.0.20' #'da2020w-0016.eastus.cloudapp.azure.com' # VM Server 
es = Elasticsearch([{'host': ES_HOST}], timeout=60000)

In [0]:
schema_structfields = [
                      StructField("_id", MapType(StringType(), StringType(), True),True), 
                       StructField("actualDelay",LongType(),True),
                       StructField("angle",DoubleType(),True),
                       StructField("anomaly",BooleanType(),True),
                       StructField("areaId",LongType(),True),
                       StructField("areaId1",LongType(),True),
                       StructField("areaId2",LongType(),True),
                       StructField("areaId3",LongType(),True),
                       StructField("atStop",BooleanType(),True),
                       StructField("busStop",LongType(),True),
                       StructField("calendar",MapType(StringType(), StringType(),True),True),
                       StructField("congestion",BooleanType(),True),
                       StructField("currentHour",LongType(),True),
                       StructField("dateType",LongType(),True),
                       StructField("dateTypeEnum",StringType(),True),
                       StructField("delay",LongType(),True),
                       StructField("direction",LongType(),True),
                       StructField("distanceCovered",DoubleType(),True),
                       StructField("ellapsedTime",LongType(),True),
                       StructField("filteredActualDelay",LongType(),True),
                       StructField("gridID",StringType(),True),
                       StructField("journeyPatternId",StringType(),True), 
                       StructField("justLeftStop",BooleanType(),True),
                       StructField("justStopped",BooleanType(),True),
                       StructField("latitude",DoubleType(),True), 
                       StructField("lineId",StringType(),True), 
                       StructField('loc',StructType([StructField('coordinates',ArrayType(DoubleType(),True),True),StructField('type',StringType(),True)]),True),
                       StructField("longitude",DoubleType(),True),  
                       StructField("poiId",LongType(),True),
                       StructField("poiId2",LongType(),True),
                       StructField("probability",DoubleType(),True),
                       StructField("systemTimestamp",DoubleType(),True),
                       StructField("timestamp",MapType(StringType(),StringType(),True)), 
                       StructField("vehicleId",LongType(),True),
                       StructField("vehicleSpeed",LongType(),True)]

schema = StructType(schema_structfields)

In [0]:

 
kafka_server = '10.0.0.30:9091'
 
# Subscribe to multiple topics
# kafka_raw_df = spark \
#   .readStream \
#   .format("kafka") \
#   .option("kafka.bootstrap.servers", kafka_server) \
#   .option("subscribe", "vehicleId_28051,vehicleId_28052") \
#   .option("startingOffsets", "earliest") \
#   .load()
 
# Subscribe to a pattern
kafka_raw_df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", kafka_server) \
  .option("subscribePattern", "vehicleId_.*") \
  .option("startingOffsets", "earliest") \
  .option("maxOffsetsPerTrigger", 5000000).load()
 
kafka_value_df = kafka_raw_df.selectExpr("CAST(value AS STRING)")
 
# schema = pickle.load(open("/dbfs/mnt/schema.pkl", "rb"))
 
stream_raw_df = kafka_value_df \
           .select(F.from_json(F.col("value"), schema=schema).alias('json')) \
           .select("json.*")
 
display(stream_raw_df)

_id,actualDelay,angle,anomaly,areaId,areaId1,areaId2,areaId3,atStop,busStop,calendar,congestion,currentHour,dateType,dateTypeEnum,delay,direction,distanceCovered,ellapsedTime,filteredActualDelay,gridID,journeyPatternId,justLeftStop,justStopped,latitude,lineId,loc,longitude,poiId,poiId2,probability,systemTimestamp,timestamp,vehicleId,vehicleSpeed
Map($oid -> 59e9dbcee45b4b2f8404f16d),0,0.0,false,18341,17,286,4585,false,226,Map($numberLong -> 1508498336000),true,11,0,WEEKDAY,589,0,0.0,0,0,"156,128",00011001,false,false,53.391149,1,"List(List(-6.261696, 53.391149), Point)",-6.261696,0,0,0.0,2.060526734E9,Map($numberLong -> 1508498336000),43110,0
Map($oid -> 59e9dc20e45b4b2f8404f78e),-589,225.0,false,18341,17,286,4585,true,226,Map($numberLong -> 1508498415000),false,11,0,WEEKDAY,0,0,0.03289490140934223,79000,0,"156,128",00010001,false,true,53.391176,1,"List(List(-6.26219, 53.391176), Point)",-6.26219,0,0,0.0,2.060608641E9,Map($numberLong -> 1508498415000),43110,1
Map($oid -> 59e9dc66e45b4b2f8404fe06),0,-2.0,false,18341,17,286,4585,true,226,Map($numberLong -> 1508498488000),false,11,0,WEEKDAY,0,0,0.0,73000,0,"156,128",00010001,false,false,53.391176,1,"List(List(-6.26219, 53.391176), Point)",-6.26219,0,0,0.0,2.060678828E9,Map($numberLong -> 1508498488000),43110,0
Map($oid -> 59eda684e45b4b2f84051b53),0,0.0,false,593,8,148,593,false,2737,Map($numberLong -> 1508746834000),false,8,0,WEEKDAY,-33,0,0.0,0,0,"125,83",056A0001,false,false,53.310365,56A,"List(List(-6.353301, 53.310365), Point)",-6.353301,0,0,0.0,2.309028923E9,Map($numberLong -> 1508746834000),43110,0
Map($oid -> 59eda697e45b4b2f84051d95),0,225.0,false,593,8,148,593,false,2737,Map($numberLong -> 1508746856000),false,8,0,WEEKDAY,-33,0,0.12503807657403349,22000,0,"125,83",056A0001,false,false,53.309453,56A,"List(List(-6.354402, 53.309453), Point)",-6.354402,0,0,0.0,2.309047557E9,Map($numberLong -> 1508746856000),43110,20
Map($oid -> 59eda6a8e45b4b2f84051fb9),0,225.0,false,593,8,148,593,false,2737,Map($numberLong -> 1508746870000),false,8,0,WEEKDAY,-33,0,0.17840279505902076,14000,0,"124,82",056A0001,false,false,53.308103,56A,"List(List(-6.355853, 53.308103), Point)",-6.355853,0,0,0.0,2.309064391E9,Map($numberLong -> 1508746870000),43110,45
Map($oid -> 59eda6f0e45b4b2f840526c7),30,225.0,false,593,8,148,593,false,2737,Map($numberLong -> 1508746943000),false,8,0,WEEKDAY,-3,0,0.05006324551055946,73000,0,"124,82",056A0001,false,false,53.307694,56A,"List(List(-6.356168, 53.307694), Point)",-6.356168,0,0,0.0,2.30913644E9,Map($numberLong -> 1508746943000),43110,2
Map($oid -> 59eda702e45b4b2f840528c7),32,-2.0,false,593,8,148,593,false,2737,Map($numberLong -> 1508746959000),false,8,0,WEEKDAY,29,0,0.0,16000,0,"124,82",056A0001,false,false,53.307694,56A,"List(List(-6.356168, 53.307694), Point)",-6.356168,0,0,0.0,2.309154375E9,Map($numberLong -> 1508746959000),43110,0
Map($oid -> 59eda71ae45b4b2f84052b4c),17,225.0,false,593,8,148,593,false,2737,Map($numberLong -> 1508746984000),false,8,0,WEEKDAY,46,0,0.019000592713650892,25000,0,"124,82",056A0001,false,false,53.307545,56A,"List(List(-6.356308, 53.307545), Point)",-6.356308,0,0,0.0,2.309178381E9,Map($numberLong -> 1508746984000),43110,2
Map($oid -> 59eda71ae45b4b2f84052bfc),0,225.0,false,593,8,148,593,false,2737,Map($numberLong -> 1508746989000),false,8,0,WEEKDAY,46,0,0.2004222273843236,5000,0,"123,81",056A0001,false,false,53.306369,56A,"List(List(-6.358594, 53.306369), Point)",-6.358594,0,0,0.0,2.309178636E9,Map($numberLong -> 1508746989000),43110,144


In [0]:
stream_raw_df=stream_raw_df[["vehicleId","timestamp","longitude",'loc',"lineId","latitude","journeyPatternId","delay","congestion","busStop","atStop","areaId3","areaId2","areaId1","areaId","actualDelay"]]

fixed_df = stream_raw_df.withColumn('timestamp', F.map_values(stream_raw_df.timestamp)[0])\
                  .withColumn('loc', stream_raw_df['loc']['coordinates'])
fixed_df = fixed_df.dropDuplicates()
fixed_df = fixed_df.na.drop(how="all")

In [0]:
fixed_df = fixed_df.withColumn('timestamp', F.to_timestamp(F.from_unixtime(fixed_df.timestamp / (1000)))).withColumn('mins_delay', fixed_df.delay / 60)\
                  .withColumn('mins_actualDelay', fixed_df.actualDelay / 60) 

In [0]:
# df_stats = (fixed_df.agg(F.stddev('delay').alias('std'),F.avg('delay').alias('mean')))
# mean = df_stats[0]['mean']
# std = df_stats[0]['std']

# display(df_stats)

In [0]:
fixed_df=fixed_df.withColumn('realHour', F.hour(fixed_df.timestamp))
fixed_df=fixed_df.withColumn("isWeekend", F.dayofweek(fixed_df.timestamp).isin([1,7]).cast("int"))
fixed_df=fixed_df.withColumn('month', F.month(fixed_df.timestamp))
fixed_df=fixed_df.withColumn('year', F.year(fixed_df.timestamp))
fixed_df=fixed_df.withColumn('date', F.to_date(fixed_df.timestamp))
fixed_df=fixed_df.withColumn("season" ,F.expr(
    """IF (month IN (3,4,5) ,"Spring", IF (month IN (6,7,8), "Summer" ,IF (month IN (9,10,11) ,"Autumn","Winter")))"""
))
fixed_df=fixed_df.withColumn("timeInDay" ,F.expr(
    """IF (realHour IN (5,6,7,8,9,10,11) ,"Morning", IF (realHour IN (12, 13, 14, 15, 16, 17), "Afternoon" ,IF (realHour IN (18, 19, 20) ,"Evening","Night")))"""
))
fixed_df=fixed_df.withColumn("isInCenter" ,F.expr(
    """IF (busStop IN (278, 281, 4724, 274, 279, 4725, 4508, 272, 277, 270, 6059, 271, 7402, 1184, 4717, 288, 289, 7591, 299, 298, 297, 302, 301, 273, 315, 7622, 334, 335, 336, 340, 317, 319, 325, 7392, 328, 345, 346, 7588, 404, 405, 406, 320, 1359, 7582, 1358, 7581, 1279, 1278, 4522, 4521, 494, 495, 792, 793) ,1, 0)"""
))

fixed_df = fixed_df.withColumn('line_num', fixed_df['journeyPatternId'][0:4])
fixed_df = fixed_df.withColumn('line_num', F.regexp_replace('line_num', '^0+', '')) 
fixed_df = fixed_df.withColumn('direction', fixed_df['journeyPatternId'][5:5][0:1])
# display(fixed_df)

In [0]:
task2_df = fixed_df[fixed_df.atStop==True]

In [0]:
#the values were calculated on the static train data
mean_delay = 3.123
std_delay = 7.0039

mean_plus_std = mean_delay+2*std_delay
mean_minus_std = mean_delay-2*std_delay

task2_df=task2_df.withColumn("null_mins_delay" ,F.expr(
    f"""IF ((mins_delay > {mean_plus_std}) or (mins_delay < {mean_minus_std}) ,null, mins_delay)"""
))
task2_df = task2_df.withColumn("null_congestion", F.expr(f"""IF ((congestion=True and mins_delay < 0) or (congestion=False and mins_delay > 10) ,null, congestion)""")) #24 mins

In [0]:
display(task2_df)

vehicleId,timestamp,longitude,loc,lineId,latitude,journeyPatternId,delay,congestion,busStop,atStop,areaId3,areaId2,areaId1,areaId,actualDelay,mins_delay,mins_actualDelay,realHour,isWeekend,month,year,date,season,timeInDay,isInCenter,line_num,direction,null_mins_delay,null_congestion
43110,2017-10-23T14:17:39.000+0000,-6.41955,"List(-6.41955, 53.277767)",27,53.277767,077A0001,-655,false,4930,true,559,139,8,559,0,-10.916666666666666,0.0,14,0,10,2017,2017-10-23,Autumn,Afternoon,0,77A,0,null,false
43110,2017-10-24T15:51:14.000+0000,-6.256362,"List(-6.256362, 53.348096)",27,53.348096,00271001,-231,false,298,true,4566,285,17,73071,20,-3.85,0.3333333333333333,15,0,10,2017,2017-10-24,Autumn,Afternoon,1,27,1,-3.85,false
43110,2017-10-24T18:07:18.000+0000,-6.316657,"List(-6.316657, 53.325633)",27,53.325633,00270004,-61,false,2099,true,3761,234,14,15045,0,-1.0166666666666666,0.0,18,0,10,2017,2017-10-24,Autumn,Evening,0,27,0,-1.0166666666666666,false
43110,2017-10-26T12:31:11.000+0000,-6.198952,"List(-6.198952, 53.38853)",27,53.38853,00271001,1221,false,4385,true,4624,288,17,4624,0,20.35,0.0,12,0,10,2017,2017-10-26,Autumn,Afternoon,0,27,1,null,null
43110,2017-10-26T10:05:44.000+0000,-6.332711,"List(-6.332711, 53.317856)",27,53.317856,00270004,359,false,2333,true,3750,234,14,15002,0,5.983333333333333,0.0,10,0,10,2017,2017-10-26,Autumn,Morning,0,27,0,5.983333333333333,false
43110,2017-10-26T17:30:28.000+0000,-6.211367,"List(-6.211367, 53.395057)",27,53.395057,00270004,262,false,1254,true,4625,288,17,4625,20,4.366666666666666,0.3333333333333333,17,0,10,2017,2017-10-26,Autumn,Afternoon,0,27,0,4.366666666666666,false
40003,2017-07-03T20:05:55.000+0000,-6.44884,"List(-6.44884, 53.356466)",67,53.356466,00670001,-132,false,3367,true,728,181,11,728,0,-2.2,0.0,20,0,7,2017,2017-07-03,Summer,Evening,0,67,0,-2.2,false
40003,2017-07-05T08:15:12.000+0000,-6.260747,"List(-6.260747, 53.352875)",120,53.352875,01200001,0,false,6004,true,4566,285,17,73067,0,0.0,0.0,8,0,7,2017,2017-07-05,Summer,Morning,0,120,0,0.0,false
40003,2017-07-05T16:50:46.000+0000,-6.263758,"List(-6.263758, 53.352343)",120,53.352343,01200002,217,false,3,true,4566,285,17,73065,35,3.6166666666666667,0.5833333333333334,16,0,7,2017,2017-07-05,Summer,Afternoon,0,120,0,3.6166666666666667,false
43121,2017-07-03T17:56:12.000+0000,-6.309663,"List(-6.309663, 53.340615)",13,53.340615,00130003,601,false,1945,true,4445,277,17,17783,0,10.016666666666667,0.0,17,0,7,2017,2017-07-03,Summer,Afternoon,0,13,0,10.016666666666667,null


In [0]:
task2_df = task2_df.withColumn('binary_congestion', F.expr("""IF (congestion=True, 1, 0)"""))
task2_df = task2_df.withColumn('null_binary_congestion', F.expr("""IF (null_congestion=True, 1, IF(null_congestion=False, 0, null))""")) 

In [0]:
#dealing with uncertain data
Complete_case_df = task2_df.dropna()

#split to test data
test_df=Complete_case_df[Complete_case_df["year"]== 2018]

df_hash = spark.read.csv('lineId_index.csv',header=True)

test_df = test_df.join(df_hash.hint("broadcast"), df_hash["lineId"] == test_df["lineId"], how='inner').drop(test_df.lineId)

In [0]:
#Preparing Data for Machine Learning
test_df=test_df.withColumn('label', F.col("isInCenter"))
test_df =test_df.withColumn("lineId_index",F.col("lineId_index").cast(IntegerType()))

stages = []

assemblerInputs = ["lineId_index",'null_mins_delay', 'longitude', 'latitude','null_binary_congestion','areaId1']
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

#Pipeline
pipeline = Pipeline(stages = stages)
pipelineModel = pipeline.fit(test_df)
test_df = pipelineModel.transform(test_df)
selectedCols = ['features'] + test_df.columns
test_df = test_df.select(selectedCols)

In [0]:
savedModel = LogisticRegressionModel.load("lrm_model_task2.model")
predictions = savedModel.transform(test_df)

In [0]:
aggDF_task2 = predictions.groupBy('label', 'prediction').agg(F.count('label').alias("count"))
display(aggDF_task2)

label,prediction,count
1,0.0,123695
0,0.0,950497


In [0]:
def foreach_batch_function(predictions, epoch_id):
  #predictions = savedModel.transform(test_df)
#   preds_df=predictions.groupBy('label', 'prediction').count()
  predictionAndTarget = predictions.select("label", "prediction")
  batch_size=predictionAndTarget.count()
  #for name in ["accuracy", "f1"]: #, "weightedPrecision", "weightedRecall"]: 
  evaluator_acc = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName="accuracy")
  evaluator_f1 = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName="f1")
  measure_acc = evaluator_acc.evaluate(predictions)
  measures["acc"].append((measure_acc,batch_size))
  acc_list.append((measure_acc,batch_size))
  measure_f1 = evaluator_f1.evaluate(predictions)
  measures["f1"].append((measure_f1,batch_size))
  f1_list.append((measure_f1,batch_size))
    #print("-"*6)
    #print(f'{name}: {measure}')
  

In [0]:
f1_list=[]
acc_list=[]
measures= defaultdict(list)

predictions.writeStream\
.format("memory") \
.foreachBatch(foreach_batch_function).start()

In [0]:
def calc_f1_acc(measures):
  test_data_size=sum([x[1] for x in measures["f1"]])
  output_f1=measures["f1"]
  output_acc=measures["acc"]
  avg_acc,avg_f1=0,0
  num_batches=len(output_f1)
  for (f1_m,batch_size),(acc_m,batch_size) in zip(output_f1,output_acc):
    avg_acc+=acc_m*(batch_size/test_data_size)
    avg_f1+=f1_m*(batch_size/test_data_size)
  print(f"avg acc is {avg_acc:.4f} and avg f1 is {avg_f1:.4f}")

In [0]:
calc_f1_acc(measures)

avg acc is 0.8852963870243671 and avg f1 is 0.8314438051412673

In [0]:
source_dest_df = spark.read.csv('source_dest_df_1.csv',header=True)
atrractions_df = spark.read.csv('atrractions_with_busStops.csv',header=True).drop("Longitude","Latitude")


In [0]:
first_joined_df=fixed_df.join(F.broadcast(source_dest_df), on=["line_num", "direction"],how="left")
display(first_joined_df) 

In [0]:
joined_df=first_joined_df.join(F.broadcast(atrractions_df),on=["busStop"],how="left")
display(joined_df)

busStop,line_num,direction,vehicleId,timestamp,longitude,loc,lineId,latitude,journeyPatternId,delay,congestion,atStop,areaId3,areaId2,areaId1,areaId,actualDelay,mins_delay,mins_actualDelay,realHour,isWeekend,month,year,date,season,timeInDay,isInCenter,sourceAndDest,source,dest,Name,AddressLocality,distance,rank
1286,9,0,44131,2017-07-03T11:31:00.000+0000,-6.265266,"List(-6.265266, 53.334337)",9,53.334337,00090001,-4,false,false,3796,236,14,15188,0,-0.06666666666666667,0.0,11,0,7,2017,2017-07-03,Summer,Morning,0,From Charlestown To Limekiln Ave.,Charlestown,Limekiln Ave.,The Bernard Shaw,Dublin City Area South,0.0014347469582949575,3
2392,9,0,44131,2017-07-03T14:43:04.000+0000,-6.291214,"List(-6.291214, 53.318935)",9,53.318935,00090001,-158,false,false,3782,236,14,3782,0,-2.6333333333333333,0.0,14,0,7,2017,2017-07-03,Summer,Afternoon,0,From Charlestown To Limekiln Ave.,Charlestown,Limekiln Ave.,null,null,null,null
4392,9,0,44131,2017-07-03T15:03:10.000+0000,-6.33445,"List(-6.33445, 53.306933)",9,53.306933,00090001,-8,false,true,3749,234,14,3749,0,-0.13333333333333333,0.0,15,0,7,2017,2017-07-03,Summer,Afternoon,0,From Charlestown To Limekiln Ave.,Charlestown,Limekiln Ave.,null,null,null,null
142,9,0,44131,2017-07-03T17:20:22.000+0000,-6.274167,"List(-6.274167, 53.39013)",9,53.39013,00090001,565,false,true,4497,280,17,4497,0,9.416666666666666,0.0,17,0,7,2017,2017-07-03,Summer,Afternoon,0,From Charlestown To Limekiln Ave.,Charlestown,Limekiln Ave.,null,null,null,null
2241,66,1,44131,2017-07-04T07:37:47.000+0000,-6.375262,"List(-6.375262, 53.355836)",26,53.355836,00661002,16,false,false,3043,189,11,3043,0,0.26666666666666666,0.0,7,0,7,2017,2017-07-04,Summer,Morning,0,From Merrion Sq. To Maynooth,Maynooth,Merrion Sq.,null,null,null,null
1474,66,1,44131,2017-07-04T10:48:45.000+0000,-6.292347,"List(-6.292347, 53.347905)",26,53.347905,00661002,105,false,true,4470,279,17,17883,20,1.75,0.3333333333333333,10,0,7,2017,2017-07-04,Summer,Morning,0,From Merrion Sq. To Maynooth,Maynooth,Merrion Sq.,null,null,null,null
406,66,1,44131,2017-07-04T11:01:35.000+0000,-6.259267,"List(-6.259267, 53.3437)",26,53.3437,00661002,0,false,false,4565,285,17,73057,0,0.0,0.0,11,0,7,2017,2017-07-04,Summer,Morning,1,From Merrion Sq. To Maynooth,Maynooth,Merrion Sq.,The KEMP Gallery,Dublin City,4.661626026481523E-4,1
406,66,1,44131,2017-07-04T11:01:35.000+0000,-6.259267,"List(-6.259267, 53.3437)",26,53.3437,00661002,0,false,false,4565,285,17,73057,0,0.0,0.0,11,0,7,2017,2017-07-04,Summer,Morning,1,From Merrion Sq. To Maynooth,Maynooth,Merrion Sq.,Gormley's Fine Art,Dublin City,4.531639078365117E-4,2
406,66,1,44131,2017-07-04T11:01:35.000+0000,-6.259267,"List(-6.259267, 53.3437)",26,53.3437,00661002,0,false,false,4565,285,17,73057,0,0.0,0.0,11,0,7,2017,2017-07-04,Summer,Morning,1,From Merrion Sq. To Maynooth,Maynooth,Merrion Sq.,Rabbie’s Small Group Tours,Dublin City,0.001024477066401041,2
406,66,1,44131,2017-07-04T11:01:35.000+0000,-6.259267,"List(-6.259267, 53.3437)",26,53.3437,00661002,0,false,false,4565,285,17,73057,0,0.0,0.0,11,0,7,2017,2017-07-04,Summer,Morning,1,From Merrion Sq. To Maynooth,Maynooth,Merrion Sq.,The Doorway Gallery,Dublin City,3.9773942312975664E-4,1


In [0]:
joined_df = joined_df.withColumn("attraction_catagory",F.when(F.col('atStop')==False,2).otherwise(F.when(F.col('Name').isNull(),0).otherwise(1)))

In [0]:
df_to_elastic=joined_df["busStop","vehicleId","line_num","direction","timestamp","loc","congestion","atStop","areaId1","mins_delay","realHour","isWeekend","month",
"year",
"date",
"season",
"timeInDay",
"isInCenter",
"sourceAndDest",
"source",
"dest",
"Name","AddressLocality",
"distance",
"rank",
"attraction_catagory"]

index="stream_joined_data"
# es.indices.delete(index=index)

In [0]:
def write_to_elastic(df_to_elastic, index, settings_with_mapping, ES_HOST):
  es.indices.create(index=index, ignore=400, body=settings_with_mapping)
  df_to_elastic.writeStream \
  .outputMode("append") \
  .queryName(f"elastic_{index}") \
  .format("org.elasticsearch.spark.sql") \
  .option("es.resource", index) \
  .option("es.nodes.wan.only","true") \
  .option("es.port","9200") \
  .option("es.nodes", ES_HOST) \
  .option('es.nodes.client.only', 'false')\
  .option("checkpointLocation", "/tmp/") \
  .start()

In [0]:
settings_with_mapping = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0,
        "refresh_interval" : -1
    },
     "mappings": {
      "properties": {
          "mins_delay" : { "type": "long" },#*
          "areaId1" : { "type": "long" },#*
          "atStop" : { "type": "boolean" },#*
          "busStop" : { "type": "long" }, #* 

        "congestion" : { "type": "boolean" },#*
          "loc" : { "type": "geo_point" },#*
          "timestamp" : { "type": "date"}, #*
          "vehicleId" : { "type": "long" },
          "realHour" : { "type": "long" },#*
          "isWeekend" : { "type": "long" },#*
          "date" : {"type":"date"},#*
          "month" : {"type": "long"}, #*
          "year" : {"type": "long"}, #*
          "season" : {"type": "keyword"},#*
          "timeInDay" : {"type": "keyword"}, #*
          "isInCenter" : {"type" : "boolean"}, #*
          "line_num" : { 'type' : 'keyword'},#*
          "sourceAndDest" : { 'type' : 'keyword'},#*
          "source" : { 'type' : 'keyword'},#*
          "dest" : { 'type' : 'keyword'},#*
          "direction" : { 'type' : 'long'},#*
          "Name" : { 'type' : 'keyword'},#*
          "AddressLocality" : { 'type' : 'keyword'},#*
          "distance": { "type": "long" },#*
          "rank": { "type": "long" },#*
          "attraction_catagory": { "type": "long" },#*

    }
     }
}
# write_to_elastic(df_to_elastic, index, settings_with_mapping, ES_HOST)
# if not es.indices.exists(index):
es.indices.create(index=index, ignore=400, body=settings_with_mapping)

df_to_elastic.writeStream \
    .outputMode("append") \
    .queryName(f"elastic_{index}") \
    .format("org.elasticsearch.spark.sql") \
    .option("es.resource", index) \
    .option("es.nodes.wan.only","true") \
    .option("es.port","9200") \
    .option("es.nodes", ES_HOST) \
    .option("checkpointLocation", "/tmp/") \
    .start()

Out[193]: <pyspark.sql.streaming.StreamingQuery at 0x7f0221fa8588>

In [0]:
task_3_df = joined_df[joined_df.atStop==True]

#split to test data
task3_test_df=task_3_df[task_3_df["year"]== 2018]
task3_test_df = task3_test_df.withColumn('binary_congestion', F.expr("""IF (congestion=True, 1, 0)"""))

task3_test_df=task3_test_df["mins_delay","areaId1","binary_congestion","loc","isInCenter","line_num","source","dest","attraction_catagory",'longitude', 'latitude'] 
task3_test_df = task3_test_df.withColumn("source", F.when(F.col('source').isNull() ,"empty").otherwise(F.col('source'))).withColumn("dest", F.when(F.col('dest').isNull() ,"empty").otherwise(F.col('dest')))


In [0]:
categoricalCols={'line_num':'line_num_index.csv','source':'source_index.csv','dest':'dest_index.csv'}
for c, c_file in categoricalCols.items():
  df_hash = spark.read.format("csv").option("header", "true").option('sep','|').load(c_file, inferSchema="true")
  df_hash=df_hash.withColumn(c+"_index",(split(F.col(c+"_index,"+c),",")[0]).cast("int")).withColumn(c,split(F.col(c+"_index,"+c),",")[1]).drop(c+"_index,"+c)
  task3_test_df = task3_test_df.join(df_hash.hint("broadcast"), df_hash[c] == task3_test_df[c], how='inner').drop(df_hash[c])

In [0]:
#Preparing Data for Machine Learning
task3_test_df=task3_test_df.withColumn('label', F.col("isInCenter"))
stages = []
assemblerInputs = ['source_index','dest_index',"line_num_index",'mins_delay', 'longitude', 'latitude','binary_congestion','areaId1']
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

#Pipeline
pipeline = Pipeline(stages = stages)
pipelineModel = pipeline.fit(task3_test_df)
task3_test_df = pipelineModel.transform(task3_test_df)
selectedCols = ['features'] + task3_test_df.columns
task3_test_df = task3_test_df.select(selectedCols)



In [0]:
savedModel_task3 = LogisticRegressionModel.load("lrm_model_task3.model")
task3_predictions = savedModel_task3.transform(task3_test_df)

In [0]:
display(task3_predictions)

In [0]:
aggDf=task3_predictions.groupBy('label', 'prediction').agg(F.count('label').alias("count"))
display(aggDf)

In [0]:
def task3_foreach_batch_function(predictions, epoch_id):
  predictionAndTarget = predictions.select("label", "prediction")
  
  batch_size=predictionAndTarget.count()
  
  evaluator_acc = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName="accuracy")
  evaluator_f1 = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName="f1")
  
  measure_acc = evaluator_acc.evaluate(predictions)
  task3_measures["acc"].append((measure_acc,batch_size))
  task3_acc_list.append((measure_acc,batch_size))
  measure_f1 = evaluator_f1.evaluate(predictions)
  task3_measures["f1"].append((measure_f1,batch_size))
  task3_f1_list.append((measure_f1,batch_size))


In [0]:
task3_f1_list=[]
task3_acc_list=[]
task3_measures= defaultdict(list)

task3_predictions.writeStream\
.format("memory") \
.foreachBatch(task3_foreach_batch_function).start()

In [0]:
calc_f1_acc(task3_measures) 

avg acc is 0.8381 and avg f1 is 0.8314